# Importing the Female Adult Fly Brain (FAFB) Connectome

In [1]:
import pymaid
source = pymaid.CatmaidInstance('https://fafb.catmaid.virtualflybrain.org/' , api_token=None)

neurons = pymaid.get_annotated("Paper: Zheng et al 2018") # Paper of interest

INFO  : Global CATMAID instance set. Caching is ON. (pymaid)
INFO  : Searching for: Paper: Zheng et al 2018 (pymaid)


### This step will possibly throw an error depending on the version of pymaid you have. We tested this with the latest version of pymaid from Github, which you can download with `pip3 install git+git://github.com/schlegelp/pymaid@master`

## Get Neuron IDs in a List for Indexing

In [2]:
skeleton_ids = [j for j in neurons["skeleton_ids"]] # Get Skeleton IDs in list

old = skeleton_ids
skeleton_ids = []

for i in range(len(old)):
    skeleton_ids.append(old[i][0])

## Import Connectome

In [3]:
connectome = pymaid.adjacency_matrix(skeleton_ids)

In [4]:
cd ..

/mnt/c/Users/casey/Documents/School/Senior Year/First Semester/NDD/2020-2021/MyFork/connectocross


In [5]:
from graph.grapgh_io import GraphIO
import networkx as nx

connectome_nx = nx.from_pandas_adjacency(connectome, create_using=nx.DiGraph)

## Add Metadata

In [6]:
count = 0
for id in connectome_nx.nodes:
    connectome_nx.nodes[id]['Name'] = neurons['name'][count]
    count += 1

    try:
        # Soma location, hemisphere
        df = pymaid.get_node_location(id)
        connectome_nx.nodes[id]['X'] = df['x'][0]
        connectome_nx.nodes[id]['Y'] = df['y'][0]
        connectome_nx.nodes[id]['Z'] = df['z'][0]
        if df['x'][0] < 550000:
            connectome_nx.nodes[id]['Side'] = 'L'
        else:
            connectome_nx.nodes[id]['Side'] = 'R'
    except:
        connectome_nx.nodes[id]['X'] = False
        connectome_nx.nodes[id]['Y'] = False
        connectome_nx.nodes[id]['Z'] = False
        connectome_nx.nodes[id]['Side'] = False

for edge in connectome_nx.edges:

    connectome_nx.edges[edge]['synapse_type'] = "chemical"

## Create `.json` File

In [7]:
G = GraphIO.dump(connectome_nx, "json_connectomes/fafb.json")